## 1. Ingesta (Capa Bronce)
# Convertimos CSV crudo a formato Delta Lake.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from delta import *
import os

# URL del Master (definida en docker-compose)

In [2]:
master_url = "spark://spark-master:7077"

# Configuración: Añadimos Delta Lake

In [3]:
builder = (
    SparkSession.builder
    .appName("Lab_SECOP_Bronze")
    .master("spark://spark-master:7077")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", "1g")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-952a6e2d-3132-4be4-8d39-8a51224b6b99;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 286ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

In [4]:
print("Leyendo CSV crudo...")

df_raw = (spark.read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load("file:/app/data/SECOP_II_Contratos_Electronicos.csv")
)

import re

def limpiar_nombre_columna(c: str) -> str:
    c = c.strip()
    # reemplaza cualquier caracter inválido para Delta por "_"
    c = re.sub(r"[ ,;{}\(\)\n\t=]+", "_", c)
    # evita dobles "__"
    c = re.sub(r"_+", "_", c)
    # evita "_" al inicio/fin
    c = c.strip("_")
    return c.lower()

# ver columnas originales
print("Columnas originales:")
print(df_raw.columns)

# renombrar
df_bronze = df_raw
for c in df_raw.columns:
    df_bronze = df_bronze.withColumnRenamed(c, limpiar_nombre_columna(c))

print("\nColumnas limpias:")
print(df_bronze.columns)


print("Escribiendo en capa Bronce (Delta)...")
output_path = "file:/app/data/lakehouse/bronze/secop"

(df_bronze.write
    .format("delta")
    .mode("overwrite")
    .save(output_path)
)

print("✅ OK - filas:", df_bronze.count())



Leyendo CSV crudo...


26/02/02 01:44:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
26/02/02 01:44:26 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/02 01:44:41 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
                                                                                

Columnas originales:
['Entidad', 'Nit Entidad', 'Departamento', 'Ciudad', 'Estado', 'Descripcion del Proceso', 'Tipo de Contrato', 'Modalidad de Contratacion', 'Justificacion Modalidad de Contratacion', 'Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato', 'Precio Base', 'Valor Total', 'Valor Pagado']

Columnas limpias:
['entidad', 'nit_entidad', 'departamento', 'ciudad', 'estado', 'descripcion_del_proceso', 'tipo_de_contrato', 'modalidad_de_contratacion', 'justificacion_modalidad_de_contratacion', 'fecha_de_firma', 'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato', 'precio_base', 'valor_total', 'valor_pagado']
Escribiendo en capa Bronce (Delta)...


26/02/02 01:44:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

✅ OK - filas: 1000


26/02/02 01:46:19 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
26/02/02 01:46:20 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce